# Confidence Intervals

Author & Instructor: Diana NURBAKOVA, PhD.

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="../styles/styles.css">

## Learning Objectives

By the end of this lesson, you will be able to:
- Define confidence intervals
- Interpret confidence intervals
- Calculate CI for mean, proportion, variance
- Calculate bootstrap CI

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_style("whitegrid")
#sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
import ipywidgets as widgets

In [ ]:
plt.rcParams['font.family'] = ['DejaVu Sans', 'Segoe UI Emoji']

In [ ]:
import sys
from pathlib import Path

# Add the "resources" directory to the path
project_root = Path().resolve().parent
resources_path = project_root / 'resources'
sys.path.insert(0, str(resources_path))

In [ ]:
from confidence_intervals import (demonstrate_the_crisis, visualize_ci_interpretation, t_distribution_viz, dissect_confidence_interval, chi_2_distribution_viz)

<div class="alert alert-info">
<h4>🎯 Today's Challenge: The AI Product Launch Crisis</h4>

**The Scenario:**
You're the lead ML engineer at a startup. Your image classification model is ready for production. 

Your boss asks: *"What's the accuracy?"*</br>
You answer: *"94.2% on our test set of 500 images."*</br>
Your boss: *"Great! I'm telling investors we have 94% accuracy."* 

But then... After deployment, real-world accuracy is only 92.5%: 

- Investors are upset: "You promised 94%, we're only seeing 92.5%!"
- Your reputation takes a hit
- The product launch credibility is damaged

The autopsy meeting:

Boss: "You said 94.2%! What happened? Why are we seeing 92.5%?"</br>
You realize: You gave a POINT ESTIMATE without uncertainty quantification.

What you should have said: *"Our accuracy is 94.2%, but with 95% confidence, the true accuracy is between 92.2% and 96.2%. We might see values as low as 92.2% in production."*

**The lesson:**
- The 92.5% we're seeing IS within the expected range
- Nothing went "wrong" - this is normal sampling variability
- But communicating only the point estimate set wrong expectations

**Today's mission**: Learn to ALWAYS quantify uncertainty
</div>
 

In [ ]:
# visualisation
demonstrate_the_crisis()

## What is a Confidence Interval?

Point estimation can be a good starting point for estimating a population parameter. 

Let's consider $\bar{X}$ as a point estimator of the population expected value $m$ which is unknown.

<center>
<img src="img/point-estimate-0.png" width="300px" alt="Point estimate of an unknown population parameter">
</center>

As the true value of $m$ is unknown, it can be smaller or greater or equal to $\bar{X}$:

<center>
<img src="img/point-estimate-1.png" width="300px" alt="Point estimate vs a true population parameter value">
</center>

> What is the error of this estimate, i.e. *how far an estimate might be from the true value*? </br>

<div class="alert alert-success">
<h4>Definition: Margin of Error</h4>

**Margin of error** (ME), $\epsilon$, is an error associated with the estimation of the parameter under study. It can be considered as the largest distance between the point estimate and the bounds of the interval that includes the value of the population parameter with a high probability.

<center>
<img src="img/point-estimate-2.png" alt="Margin of error" width="300px">
</center>

Two components:
1. The first reflects desired confidence level
2. The second measures precision (sampling variability)

**Key Properties:**
- Is symmetric: the interval extends equally in both directions (i.e. $\text{Estimate} - \epsilon$ and $\text{Estimate} + \epsilon$)

<center>
<img src="img/point-estimate-3.png" alt="Symmetry of the margin of error" width="300px">
</center>

- Depends on confidence level: More confidence → Wider interval → Less precise
- Decreases with sample size $ME \propto \frac{1}{\sqrt{n}}$ (i.e. to cut ME in half, need 4× the sample size)
- Has same units as estimate 


</div>

| Correct Interpretations | Incorrect Interpretations|
|---|---|
|"Our estimate could be off by about ±ME"| "ME tells us if our estimate is biased" (NO - ME is about precision, not bias)|
| "The true value is likely within ME of our estimate"|"ME is the maximum possible error" (NO - it's tied to confidence level)
| "ME quantifies the precision of our estimate"||
|"Smaller ME = more precise estimate"||

<div class="alert alert-success">
<h4> Definition: Confidence Interval</h4>

A **confidence interval (CI)** is a range of plausible values for an unknown parameter.

Let $\beta$ be a **level of confidence** (LC), i.e. a value that reflects the certainty that the interval estimate contains the value of the population parameter, e.g. 95% ($\beta = 0.95$), 99% ($\beta = 0.99$).

**Notation:**
- $\theta$ = unknown parameter (e.g., true accuracy, true mean)
- $[L, U]$ = confidence interval ($L$ = Lower, $U$ = Upper bounds)
- $1 - \alpha$ = confidence level (typically 0.95 for 95%)

**Formal Definition:**
A $(1-\alpha) \times 100\%$ confidence interval satisfies:
$$P(L \leq \theta \leq U) = 1 - \alpha$$

or in other words:
$$P(\theta \in [L, U]) = \beta$$

This is a random interval because $L$ and $U$ are random variables.

The values $L$ and $U$ are called **critical values**, i.e. the number of standard errors we need to go out from our estimate to capture the desired confidence level. Critical values mark the boundaries of the middle $(1-\alpha)$ region in distribution.

**Key components:**
- **Point estimate** $\hat{\theta}$: our best guess
- **Margin of error** $\epsilon$: how far we might be off
- **Interval**: $[\hat{\theta} - \epsilon, \hat{\theta} + \epsilon]$

**Common confidence levels:**
- 90% confidence: $\alpha = 0.10$
- 95% confidence: $\alpha = 0.05$ (most common)
- 99% confidence: $\alpha = 0.01$

</div>

<center>
<img src="img/ci.png" alt="Confidence Interval" width="800px">
</center>

In [ ]:
visualize_ci_interpretation()

<div class="alert alert-danger">
<h4>CI Interpretation: What CI Really Means</h4>

**WRONG interpretation (don't say this!):**</br>
"There's a 95% probability that $\theta$ is in [L, U]"

**CORRECT interpretation:**</br>
"If we repeated this procedure many times, 95% of the intervals would contain $\theta$"

**Why the difference matters:**
- $\theta$ is FIXED (not random) - it's the true value
- The INTERVAL is random (depends on random data)
- Confidence is about the PROCEDURE, not about $\theta$

**Analogy:** 
Think of a fishing net with 95% catch rate. Each time you cast:
- The fish location ($\theta$) is fixed
- Your net location (CI) varies
- 95% of casts will catch the fish

</div>

<div class="alert alert-warning">
<h4>💡 Why This Interpretation Matters in ML</h4>

*Example*: Model Deployment Decision

You report: "Accuracy = 94% ± 2% (95% CI)"</br>
Boss asks: "So there's a 95% chance accuracy is between 92-96%?"</br>
You (**now**): "Not quite. It means: if we tested the model on many different test sets and computed a CI each time, 95% of those intervals would contain the true accuracy. For THIS specific test set, the true accuracy is either in [92%, 96%] or it isn't - but our methodology is reliable 95% of the time."

*Practical implication*: You can't say "probably between 92-96%". You say "I'm confident it's between 92-96%, based on a method that's right 95% of the time."
</div>

In [ ]:
dissect_confidence_interval()

<div class="alert alert-example">
<h4>Calculated Example: CI</h4>

According to a survey of 600 individuals, people sleep an average of 10.5 hours per night. Construct a 98% confidence interval with a margin of error of 1.3 hours. 

<h5>Solution:</h5>

Let $n = 600$, $\bar{X} = 10.5$, $\epsilon = 1.3$, $\beta = 0.98$, $\alpha = 1 - \beta = 1 - 0.98 = 0.02$. 

$$\begin{array}{l}\bar{X} + \epsilon = 10.5 + 1.3 = 11.8 \\ \bar{X} - \epsilon = 10.5 - 1.3 = 9.2 \end{array}$$

A 98% confidence interval is: $9.2 \leq \mu \leq 11.8$ or $\mu \in [9.2, 11.8]$

<center>
<img src="img/ci-ex0.png" width="300px" alt="CI of the mean: 9.2 <= mu <= 11.8">
</center>

</div>



## Confidence Interval for the Mean

**Setup:**
- Sample: $X_1, X_2, ..., X_n$
- Sample mean: $\bar{X}$
- Population variance $\sigma^2$ is unknown
- Sample standard deviation: $s$
- Population mean estimator: $\hat{\mu} = \bar{X}$

**Want**: CI for population mean $\mu$

Here's a workflow to find a confidence interval for population mean:

<center>
<img src="img/ci-mean-workflow.png" width="800px" alt="Workflow for finding CI for population mean">
</center>

<div class="alert alert-success">
<h4>Formula: CI for Mean</h4>

$$\bar{X} - t_{\nu; \alpha/2} \leq \mu \leq \bar{X} + t_{\nu; \alpha/2}$$

</div>

> What is $t_{\nu; \alpha/2}$? </br>

### $t$-Distribution

In practice, we usually don't know $\sigma$ (population standard deviation), but we have a sample. So, we can compute sample mean $\bar{X}$ and sample std $s$. 

When we replace $\sigma$ with $s$, the statistic $$t = \frac{\bar{X} - \mu}{s/\sqrt{n}}$$ does NOT follow a normal distribution. Instead, it follows a **$t$-distribution** with $(n-1)$ degrees of freedom, where $n$ is the sample size.

<div class="alert alert-success">
<h4>Definition: Student's t-distribution</h4>

**Student's $t$-distribution** $t_\nu$ is a continuous probability distribution that generalises the standard normal distribution. But $t_\nu$ has heavier tails, and the amount of probability mass in the tails is controlled by the parameter $\nu$.

$$f(t) = \frac{\Gamma\bigg(\frac{\nu + 1}{2}\bigg)}{\sqrt{\pi\nu}\Gamma\bigg(\frac{\nu}{2}\bigg)}\bigg(1 + \frac{t^2}{\nu}\bigg)^{-(\nu + 1)/2}$$

where $\nu$ is the number of degrees of freedom, and $\Gamma$ is the gamma function.

**Key properties:**
- Symmetric around 0 (like standard normal distribution)
- Bell-shaped (like standard normal distribution)
- Heavier tails than normal (more probability in extremes)
- As degrees of freedom $\nu \rightarrow \infty$, t-distribution → standard normal

</div>

> Why heavier tails? </br>

- Using $s$ instead of $\sigma$ adds extra uncertainty
- Small samples: $s$ can vary a lot from $\sigma$
- Need wider intervals to maintain confidence level

> Degrees of Freedom: Why $\nu = (n-1)$? </br>

- We use sample mean $\bar{X}$ to compute $s$ -> This 'uses up' one degree of freedom
- Only $(n-1)$ values are 'free' to vary

> What effect of degrees of freedom? </br>

1. Small df (e.g. $df = 2$):

- Very heavy tails
- Much wider than normal
- Large uncertainty

2. Medium df (e.g. $df = 10$):

- Moderate tails
- Noticeably wider than normal
- Moderate uncertainty

3. Large df (e.g. $df = 30+$)

- Close to normal
- Nearly identical to $z$-distribution
- Uncertainty dominated by sample size, not estimation

<center>
<img src="img/t-dist.png" width="600px" alt="t-distribution">
</center>


In [ ]:
t_distribution_viz()

<div class="alert alert-warning">
<h4>💡 Key Insight: t-distribution</h4>

1. Use t-distribution when σ is unknown (almost always)
2. $t$ has heavier tails than normal (accounts for extra uncertainty)
3. Degrees of freedom = $n - 1$ for single sample mean
4. As sample size increases, t → normal
5. Rule of thumb: $df \geq 30$ means t ≈ normal
6. Always safer to use $t$ than $z$ in practice

</div>


You can use [`scipy.stats.t`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) implementation of this distribution.

<div class="alert alert-exercise">
<h4>Calculated Example: Finding Critical <em>t</em> Value</h4>

Find $t_{\nu;\alpha/2}$ if the sample size is $n=13$ and the confidence level is $\beta = 95\%$. 

<h5>Solution:</h5>

Degrees of freedom: $$\nu = n - 1 = 13 - 1 = 12$$

$$\alpha = 1 - \beta = 1 - 0.95 = 0.05 \Rightarrow \alpha/2 = 0.025$$

<center>
<img src="img/ci-t-mean-1.png" width="600px" alt="Critical values of t-distribution">
</center>

$$t_{\nu=12, \alpha/2 = 0.025} = 2.179$$

</div>



In [ ]:
n = 13 # sample size
df = n - 1 # df = 12
# OPTION 1
# critical values
stats.t.interval(confidence=0.95, df=df)

In [ ]:
# OPTION 2
conf = 0.95 
alpha = 1 - conf 
t_critical = stats.t.ppf(1 - alpha/2, df=df)
print(t_critical)

### Finding CI for Mean using $t$-distribution

<div class="alert alert-exercise">
<h4>Calculated Exercise: Build a CI for Expectation</h4>

A machine learning engineer is monitoring the inference latency of a deployed recommendation model. From a sample of 4 peak-hour batches, the average response time is 49 milliseconds with a standard deviation of s'ₙ = 15 ms. Find the 99% confidence interval for the true mean inference latency across all peak-hour periods.

<h5>Solution:</h5>

What is known: $n = 4$, $\beta = 0.99$, $\bar{X} = 49$, $s' = 15$

Hence:

$$\nu = n - 1 = 4 - 1 = 3$$
$$\alpha = 1 - \beta = 1 - 0.99 = 0.01 \Rightarrow \alpha/2 = 0.005$$

$$t_{\nu=3; \alpha/2 = 0.005} = 5.841$$

$$\epsilon = t_{\nu=3; \alpha/2 = 0.005} \times \frac{s'}{\sqrt{n}} = 5.841\times \frac{15}{\sqrt{4}} \approx 43.80$$

$$\begin{array}{l}\bar{X} - \epsilon = 49 - 43.80 = 5.20\\\bar{X} + \epsilon = 49 + 43.80 = 92.80\end{array}$$

$$5.20 \leq \mu \leq 92.80$$

The interval is very wide. One might question its usefulness. Perhaps it would be worth adding more data to reduce the interval.

</div>


In [ ]:
n = 4
df = n - 1
conf = 0.99
stats.t.interval(confidence=conf, df=df)

## Confidence Interval for the Proportion

Recall that **proportion** is a fraction of the population / sample that has a certain characteristic.

**Setup:**
- Sample: $X_1, X_2, ..., X_n$
- Sample proportion: $\hat{p}$
- Sample size $n$
- Population variance $\sigma^2$ is unknown
- Sample standard deviation: $s$
- Population proportion estimator: $\hat{p}$

**Want**: CI for population proportion $p$

**Conditions for normal approximation** (i.e. large samples where CLT applies):
- $n\times \hat{p} \geq 10$
- $n\times (1 - \hat{p}) \geq 10$

Here's a workflow to find a confidence interval for population proportion:

<center>
<img src="img/ci-proportion-workflow.png" width="800px" alt="Workflow for CI for proportion">
</center>

<div class="alert alert-success">
<h4>Formula: CI for Proportion</h4>

$$\hat{p} - z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} \leq p \leq \hat{p} + z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}}$$

</div>

> What is $z_{\alpha/2}$? </br>

$z_{\alpha/2}$ is a critical value of $z$. To find it, we use the Standard Normal distribution $\mathcal{N}(0,1)$.

Let's consider the Standard Normal distribution. The shaded region between $-z_\beta$ and $z_\beta$ corresponds to the confidence level $\beta$:

<center>
<img src="img/ci-prop-z-score.png" width="600px" alt="Normal Distribution with critical values of $z_\beta$ and $-z_\beta$">
</center>

The remaining area under the curve corresponds to $\alpha = 1 - \beta$, therefore, the area under the curve of each part (1) to the left to $-z_\beta$ and (2) to the right to $z_\beta$ is equal to $\frac{1 - \beta}{2} = \alpha/2$:

<center>
<img src="img/ci-prop-z-score-2.png" width="600px" alt="Normal Distribution with shaded area $-z_\beta$ and $>z_\beta$">
</center>

Putting all together:

<center>
<img src="img/ci-prop-z-score-3.png" width="600px" alt="Normal Distribution with critical values $z_\beta$ and $-z_\beta$">
</center>

To find the value of $z_{\alpha/2}$, we can use [`scipy.stats.norm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html):

In [ ]:
# option 1
conf = 0.95 # confidence level
stats.norm.interval(confidence=0.95)

In [ ]:
# option 2
beta = 0.95
alpha = 1 - beta
z_critical = stats.norm.ppf(1 - alpha/2)
print(z_critical)

Note that the values of $z_{\alpha/2}$ are dependent only on the confidence level and are independent of the problem. The most common values:

|Level of Confidence, $\beta$| $z_\beta$ |
|:---:|:----|
|0.80|1.2816|
|0.85|1.4395|
|0.90|1.6449|
|0.95|1.9600|
|0.98|2.3263|
|0.99|2.5758|

In [ ]:
confs = [0.8, 0.85, 0.9, 0.95, 0.98, 0.99]

print("| Level of Confidence | z_beta |")
print("_"*32)
for c in confs:
    z_c = stats.norm.interval(confidence=c)[1]
    print(f"|{c:19.2f}  | {z_c:.4f} |")
print("_"*32)

<div class="alert alert-exercise">
<h4>Calculated Exercise: Build a CI for Proportion</h4>

A spam classifier was tested on 200 emails and correctly classified 174.

Tasks:

1. Compute the point estimate for accuracy
2. Compute the standard error
3. Find the 95% critical value
4. Construct a 95% confidence interval
5. Interpret the result
6. How would the CI change if:
    - You tested on 2000 emails instead? (10x more data)
    - You wanted 99% confidence instead?
7. If your company requires 85% accuracy, can we deploy this model?

</div>

<h5>Solution:</h5>

1. Compute the point estimate for accuracy

Here, we are dealing with proportion of correctly classified emails:

$$\hat{p} = 174 / 200 = 0.87$$

Check the conditions for normal approximation:

- $n\times \hat{p} = 200 \times 0.87 = 174 \geq 10$
- $n\times (1 - \hat{p}) = 200 \times 0.13 = 26 \geq 10$

Both conditions are satisfied, so the normal approximation is valid.

2. Compute the standard error 

$$SE = \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = \sqrt{\frac{0.87 \times (1 - 0.87)}{200}} = 0.238$$

3. Find the 95% critical value

For 95% confidence level $z_\beta = 1.96$

4. Construct a 95% confidence interval

$$\begin{array}{l}\hat{p} - z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = 0.87 - 1.96 \times 0.238 = 0.823\\ \hat{p} + z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} =  0.87 + 1.96 \times 0.238 = 0.917\end{array}$$

5. Interpret the result

*We are 95% confident that the true spam detection accuracy is between 82.3% and 91.7%. In production, we should expect accuracy in this range.*

6. How would the CI change if you tested on 2000 emails instead? (10x more data)

The estimate will be modified: $\hat{p} = 1740 / 2000 = 0.87$

SE will also be affected: $SE = \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = \sqrt{\frac{0.87 \times (1 - 0.87)}{2000}} = 0.0238$

The critical value remains the same, i.e. $z_{0.95} = 1.96$.

Margin of error: $\epsilon = z_{0.95}\times \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = 1.96\times 0.0238 = 0.0147$

Hence, CI of 95% for accuracy:
$$\begin{array}{l}\hat{p} - z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = 0.87 - 0.0147 = 0.855\\ \hat{p} + z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} =  0.87 + 0.0147 = 0.885\end{array}$$


*Observation*: Larger sample → Smaller SE → Smaller margin → Narrower CI

7. How would the CI change if you wanted 99% confidence instead?

We need to update the critical value:

$$z_{0.99} = 2.5758$$

The margin of error is then given by:

$$\epsilon = z_{0.99} \times SE = 0.0613$$

CI of 99% for accuracy:
$$\begin{array}{l}\hat{p} - z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = 0.87 - 0.0613 = 0.809\\ \hat{p} + z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} =  0.87 + 0.0613 = 0.931\end{array}$$

*Observation:*  Higher confidence → Larger critical value → Wider interval

8. Decision making

- Lower bound of 95% CI: 82.3% 
- Required accuracy: 85%
    
As the lower bound is below the required accuracy, the deployment is risky. The model doesn't meet the requirements.

In [ ]:
n = 200
success = 174
conf = 0.95

p_hat = success/n
print(f"Estimator: {p_hat}")
se = np.sqrt(p_hat * (1 - p_hat) / n)
print(f"Standard error: {se}")
z_c = stats.norm.interval(confidence=conf)[1]
print(f"Critical value (95%): {z_c}")
margin_error = z_c * se
print(f"Margin of error: {margin_error}")
lower = p_hat - margin_error
upper = p_hat + margin_error  
print(f"95% CI for accuracy: [{lower:.3f}; {upper:.3f}]")

print("\nn=200 -> n=2000")
n_2000 = 2000
p_hat_2000 = success*10/n_2000
print(f"Estimator: {p_hat_2000}")
se_2000 = np.sqrt(p_hat_2000 * (1 - p_hat_2000) / n_2000)
print(f"Standard error: {se}")
z_c = stats.norm.interval(confidence=conf)[1]
print(f"Critical value (95%): {z_c}")
margin_error_2000 = z_c * se_2000
print(f"Margin of error: {margin_error_2000}")
lower_2000 = p_hat_2000 - margin_error_2000
upper_2000 = p_hat_2000 + margin_error_2000  
print(f"95% CI for accuracy: [{lower_2000:.3f}; {upper_2000:.3f}]")



print("\n95% -> 99%")
z_99 = stats.norm.interval(confidence=0.99)[1]
print(f"Critical value (99%): {z_99}")
margin_error_99 = z_99 * se
print(f"Margin of error: {margin_error_99}")
lower_99 = p_hat - margin_error_99
upper_99 = p_hat + margin_error_99  
print(f"95% CI for accuracy: [{lower_99:.3f}; {upper_99:.3f}]")



In [ ]:
def viz_ci_spam_detection():
    """Visualisation for spam detection CI exercise"""
        
    # Given data
    n = 200
    n_correct = 174
    p_hat = n_correct / n
    confidence_level = 0.95
    alpha = 1 - confidence_level
    
    # standard error
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    # critical value
    z_critical = stats.norm.ppf(1 - alpha/2)
    # margin of error
    margin = z_critical * se
    # confidence interval 
    ci_lower = p_hat - margin
    ci_upper = p_hat + margin
    
    # required accuracy for decision making    
    required_accuracy = 0.85
    
    if ci_lower >= required_accuracy:
        decision = "YES ✓"
        reason = f"Even at the lower bound ({ci_lower:.1%}), we meet requirements"
    else:
        decision = "RISKY"
        reason = f"Lower bound ({ci_lower:.1%}) is below requirement ({required_accuracy:.0%})"
    
    # What sample size would we need?
    # ME = z * sqrt(p(1-p)/n)
    # We want: p - ME >= required_accuracy
    # ME <= p - required_accuracy
    # z * sqrt(p(1-p)/n) <= p - required_accuracy
    # n >= z^2 * p(1-p) / (p - required_accuracy)^2
    
    desired_margin = p_hat - required_accuracy
    n_needed = (z_critical**2 * p_hat * (1-p_hat)) / (desired_margin**2)
        
    # Visualization
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.4, wspace=0.3)
    
    # Plot 1: Sampling distribution with CI
    ax1 = fig.add_subplot(gs[0, :])
    
    p_range = np.linspace(p_hat - 4*se, p_hat + 4*se, 500)
    sampling_dist = stats.norm.pdf(p_range, p_hat, se)
    
    ax1.plot(p_range, sampling_dist, 'b-', linewidth=3, label='Sampling distribution')
    
    # Shade CI
    ci_range = p_range[(p_range >= ci_lower) & (p_range <= ci_upper)]
    ci_density = stats.norm.pdf(ci_range, p_hat, se)
    ax1.fill_between(ci_range, 0, ci_density, alpha=0.3, color='green',
                     label='95% CI region')
    
    # Mark important values
    ax1.axvline(p_hat, color='blue', linewidth=3, linestyle='-',
               label=f'Sample accuracy: {p_hat:.1%}')
    ax1.axvline(ci_lower, color='red', linewidth=2, linestyle='--',
               label=f'Lower bound: {ci_lower:.1%}')
    ax1.axvline(ci_upper, color='red', linewidth=2, linestyle='--',
               label=f'Upper bound: {ci_upper:.1%}')
    
    if required_accuracy > 0:
        ax1.axvline(required_accuracy, color='purple', linewidth=3, linestyle=':',
                   label=f'Required: {required_accuracy:.0%}')
    
    ax1.set_xlabel('Accuracy', fontsize=12)
    ax1.set_ylabel('Density', fontsize=12)
    ax1.set_title(f'95% Confidence Interval for Spam Detection Accuracy\n' +
                  f'Sample: {n} emails, {n_correct} correct, Accuracy: {p_hat:.1%} ± {margin:.1%}',
                  fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11, loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Different confidence levels
    ax2 = fig.add_subplot(gs[1, 0])
    
    for conf, color in zip([0.90, 0.95, 0.99], ['blue', 'green', 'red']):
        alpha_temp = 1 - conf
        z_temp = stats.norm.ppf(1 - alpha_temp/2)
        me_temp = z_temp * se
        lower_temp = p_hat - me_temp
        upper_temp = p_hat + me_temp
        
        ax2.barh([conf], [upper_temp - lower_temp], left=[lower_temp],
                height=0.02, alpha=0.6, color=color,
                label=f'{conf:.0%} CI: [{lower_temp:.3f}, {upper_temp:.3f}]')
    
    ax2.axvline(p_hat, color='black', linewidth=3, linestyle='-',
               label=f'Sample: {p_hat:.3f}')
    
    ax2.set_xlabel('Accuracy', fontsize=12)
    ax2.set_ylabel('Confidence Level', fontsize=12)
    ax2.set_title('CIs for Different Confidence Levels\n(Higher confidence → wider interval)',
                 fontsize=13, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Plot 3: Sample size effect
    ax3 = fig.add_subplot(gs[1, 1])
    
    n_range = np.arange(50, 2501, 50)
    margins = [z_critical * np.sqrt(p_hat * (1-p_hat) / n_val) for n_val in n_range]
    
    ax3.plot(n_range, margins, 'g-', linewidth=3)
    ax3.scatter([n], [margin], s=300, color='red', marker='*',
               edgecolors='darkred', linewidths=2, zorder=5,
               label=f'Current: n={n}')
    
    ax3.set_xlabel('Sample Size (n)', fontsize=12)
    ax3.set_ylabel('Margin of Error', fontsize=12)
    ax3.set_title('Effect of Sample Size on Precision\n(Margin decreases as √n)',
                 fontsize=13, fontweight='bold')
    ax3.legend(fontsize=11)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Decision framework
    ax4 = fig.add_subplot(gs[2, :])
    ax4.axis('off')
    
    decision_text = f"""
    DECISION FRAMEWORK
    ═══════════════════════════════════════════════════════════════════════════    
    TEST RESULTS:
    • Sample size: n = {n} emails
    • Correct classifications: {n_correct}
    • Sample accuracy: {p_hat:.1%}
    • 95% CI: [{ci_lower:.1%}, {ci_upper:.1%}]
    • Margin of error: ±{margin:.1%}    
    ───────────────────────────────────────────────────────────────────────────    
    BUSINESS REQUIREMENT:
    • Minimum accuracy: {required_accuracy:.0%}    
    ───────────────────────────────────────────────────────────────────────────
    ANALYSIS:
    • Point estimate ({p_hat:.1%}) exceeds requirement ✓
    • Lower CI bound ({ci_lower:.1%}) {'≥' if ci_lower >= required_accuracy else '<'} requirement {'✓' if ci_lower >= required_accuracy else '✗'}
    
    DECISION: {decision}
    
    REASONING: {reason}    
    ───────────────────────────────────────────────────────────────────────────    
    RECOMMENDATIONS:
    
    1. CONSERVATIVE APPROACH (Recommended):
       Use lower bound of CI for decisions
       → Protects against worst-case (with 95% confidence)
    
    2. IF DEPLOYING:
       • Set monitoring alerts at {ci_lower:.1%}
       • Expect performance in range [{ci_lower:.1%}, {ci_upper:.1%}]
       • Re-evaluate if accuracy drops below {ci_lower:.1%}
    
    3. TO INCREASE CONFIDENCE:
       • Collect more data (target: n ≥ {np.ceil(n_needed):.0f})
       • This will narrow CI and reduce uncertainty    
    ═══════════════════════════════════════════════════════════════════════════
    """
    
    ax4.text(0.5, 0.1, decision_text, fontsize=9, family='monospace',
            verticalalignment='center', horizontalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.4))
    
    plt.suptitle('Spam Detection: Complete CI Analysis & Decision Framework',
                fontsize=16, fontweight='bold')
    plt.show()
    
    return {
        'n': n,
        'p_hat': p_hat,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'margin': margin,
        'meets_requirement': ci_lower >= required_accuracy
    }

# Run solution
results = viz_ci_spam_detection()

## Confidence Interval for the Variance

What about variance?
- How much variability is in our data?
- Is our model's performance consistent?
- What's the uncertainty in our variance estimate?

**Setup:**
- Sample: $X_1, X_2, ..., X_n$
- Sample proportion: $\hat{p}$
- Sample size $n$
- Population variance $\sigma^2$ is unknown
- Sample standard deviation: $s$
- Population variance estimator: $\hat{\sigma}^2 = s^2$

**Want**: CI for population variance $\sigma^2$


Here's a workflow to find a confidence interval for population variance:

<center>
<img  src="img/ci-variance-workflow.png" width="800px" alt="Workflow for CI for Variance">
</center>
</br>

The shaded region corresponds to the confidence level $\beta$:

<center>
<img src="img/chi2-0.png" width="600px" alt="Asymmetric intervals">
</center>

**Note: ASYMMETRIC interval**
- Lower bound uses UPPER chi-square critical value
- Upper bound uses LOWER chi-square critical value

<center>
<img src="img/chi2-1.png" width="600px" alt="Asymmetric intervals">
</center>

<center>
<img src="img/chi2-2.png" width="600px" alt="Asymmetric intervals">
</center>

Upper critical value $\chi_{\nu; \alpha/2}^2$:
<center>
<img src="img/chi2-3.png" width="600px" alt="Asymmetric intervals">
</center>

Lower critical value $\chi_{\nu;1-\alpha/2}^2$:
<center>
<img src="img/chi2-4.png" width="600px" alt="Asymmetric intervals">
</center>

This is because:
- Variance can't be negative
- Large values are more likely than small values
- $\chi^2$ is right-skewed
    


<div class="alert alert-success">
<h4>Formula: CI for Variance</h4>

$$\frac{\nu s^2}{\chi_{\nu; \alpha/2}^2}\leq \sigma^2 \leq \frac{\nu s^2}{\chi_{\nu; 1-\alpha/2}^2}$$

</div>

> What is $\chi_{\nu; \alpha/2}$ and $\chi_{\nu; 1-\alpha/2}$?</br>

### $\chi^2$ Distribution 

<div class="alert alert-success">
<h4>Definition: Chi-Squared Distribution</h4>

**$\chi^2$-distribution** with $k$ degrees of freedom is the distribution of a sum of the squares of $k$ independent standard normal random variables.

If $Z_1, Z_2, ..., Z_k$ are independent, standard normal random variables, then the sum of their squares $$X = \sum_{i=1}^k Z_i^2 \sim \chi_{\nu=k}^2$$

Its PDF is given by:

$$f(x, k) = \left\{ \begin{array}{ll} \frac{x^{k/2 - 1} e^{-x/2}}{2^{k/2}\Gamma(\frac{k}{2})}, & x > 0\\ 0, & otherwise \end{array} \right.$$

where $\Gamma\bigg(\frac{k}{2}\bigg)$ is the gamma function.


*Intuition:*
- Sum of squared normal variables
- Always positive (squares are always ≥ 0)
- Right-skewed (can't be negative, but can be large)

For a sample from $N(\mu, \sigma^2)$:
the statistic $\frac{(n - 1)s^2}{\sigma^2} \sim \chi_{\nu=(n-1)}^2$, where $n$ is the sample size, $s^2$ is sample variance, $\sigma^2$ is true population variance. This is why we use $\chi^2$ for inference about variance.

**Key properties:**
1. Shape:
   - Right-skewed (**not symmetric**)
   - Always positive (domain: $[0, \infty)$)
   - Approaches normal as df increases

2. Parameters:
   - Only ONE parameter: degrees of freedom ($df$ or $k$)
   - Denoted: $\chi^2(k)$ or $\chi^2_k$

3. Mean and Variance:
   - $Mean = k$ (degrees of freedom)
   - $Variance = 2k$
   - Standard deviation = $\sqrt{2k}$
</h4>

> Degrees of Freedom: What Effect? </br>

- Small $df$: Very skewed, peak near 0
- Medium $df$: Moderately skewed
- Large $df$: Nearly symmetric (approaches normal)

<center>
<img src="img/chi-2.png" width="600px" alt="Chi-squared distribution">
</center>

> Degrees of Freedom: Why $\nu = n - 1$? </br>

For sample variance $s^2$: $df = n - 1$:
- We use $\bar{X}$ to compute $s^2$
- This 'uses up' one degree of freedom"
- Same reason as for $t$-distribution



| Property | Normal | $t$ | $\chi^2$ |
|---:|:---:|:---:|:---:|
| **Symmetry**    | Symmetric    | Symmetric    | Right-skewed |
| **Domain**      | $(-\infty, \infty)$      | $(-\infty, \infty)$      | $[0, \infty)$       |
| **Parameters**  | $\mu$, $\sigma$         | $df$           | $df$           |
| **Used for**    | Mean ($\sigma$ known) and Proportion | Mean ($\sigma$ unknown) | Variance     |
| **Shape vs df** | N/A          | Converges    | Converges |

In [ ]:
# visualisation
chi_2_distribution_viz()

You can use [`scipy.stats.chi2`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html).

## Finding CI for Variance using $\chi^2$

<div class="alert alert-exercise">
<h4>Calculated Example: CI for Variance</h4>

A data scientist is evaluating the consistency of a neural network's confidence scores for spam detection. From 15 test batches, the variance in predicted probabilities for legitimate emails is $s^2 = 3.4$. Find the 95% confidence interval for the true variance in confidence scores across all legitimate emails.

<h5>Solution:</h5>

What is known: $n = 15$, $\beta = 0.95$, $s^2 = 3.4$.

Hence:

$$\nu = n - 1 = 15 - 1 = 14$$

$$\alpha = 1 - \beta = 1 - 0.95 = 0.05 \Rightarrow \begin{array}{l}\alpha/2 = 0.025\\1 - \alpha/2 = 1 - 0.025 = 0.975\end{array}$$

$$\begin{array}{l}\chi_{\nu; \alpha/2}^2 = \chi_{14; 0.025}^2 = 26.12 \\\chi_{\nu; 1-\alpha/2}^2 = \chi_{14; 0.975}^2 = 5.63\end{array}$$

Therefore, the 95% confidence CI is given by:

$$\frac{\nu s^2}{\chi_{\nu; \alpha/2}^2}\leq \sigma^2 \leq \frac{\nu s^2}{\chi_{\nu; 1-\alpha/2}^2}$$

$$\frac{14\times 3.4}{26.12}\leq \sigma^2 \leq \frac{14 \times 3.4}{5.63}$$

$$1.82 \leq \sigma^2 \leq 8.46$$

</div>

In [ ]:
n = 15 
df = n - 1
conf = 0.95
# OPTION 1
stats.chi2.interval(confidence=conf, df=df)

In [ ]:
alpha = 1 - conf
# OPTION 2
chi2_lower = stats.chi2.ppf(alpha/2, df=df)
chi2_upper = stats.chi2.ppf(1 - alpha/2, df=df)
print(f"Chi-2 lower: {chi2_lower}")
print(f"Chi-2 upper: {chi2_upper}")

<div class="alert alert-primary">
<h4>🤖 ML Applications</h4>

- Model stability assessment
- Risk quantification
- Quality control (performance variability)
- Outlier detection thresholds
- Uncertainty estimation

</div>

<div class="alert alert-exercise">
<h4>QUESTION: Finding Sample Size</h4>

A product team is evaluating whether to deploy a new recommendation algorithm to production. Initial testing shows that approximately 78% of users click on at least one recommended item. To build a 98% confidence interval with a margin of error of 5% for the true click-through rate, how many user sessions should be included in the A/B test?

</div>

## Constructing Confidence Intervals: The Pivot Method

All the CIs we've seen follow the same general approach called the **pivot method**:

1. Find a **pivot**: a function of data and parameter with known distribution
2. Use the pivot to "invert" and isolate the parameter

**Examples:**

- **For mean** ($t$-distribution): $Pivot = (\bar{X} - \mu)/(s/\sqrt{n}) \sim t(n-1)$
  → Leads to: $CI = \bar{X} \pm t(\alpha/2) \times s/\sqrt{n}$

- **For proportion** (normal distribution): $Pivot = (\hat{p} - p)/SE \approx N(0,1)$ 
  → Leads to: $CI = \hat{p} \pm z(\alpha/2) \times SE$

- **For variance** ($\chi^2-distribution$): $Pivot = (n-1)s^2/\sigma^2 \sim \chi^2(n-1)$
  → Leads to: $CI = [(n-1)s^2/\chi^2_{\alpha/2}, (n-1)s^2/\chi^2_{1-\alpha/2}]$

**Key insight:** The pivot's distribution doesn't depend on the unknown parameter - that's what makes this work.

## Bootstrap CI

What if:

- You don't know the distribution?
- No closed-form formula exists?
- Sample size is too small for CLT?
- You're estimating a complex statistic (median, 90th percentile, etc.)?

**Solution: BOOTSTRAP**

<div class="alert alert-success">
<h4>Definition: CI using Bootstrap</h4>

**Core idea**: "The sample approximates the population"

1. Resample from your data (with replacement) using bootstrap
2. Compute statistic $\hat{\theta}^*$ on each resample
3. Repeat a certain number of times (e.g. $n_{boot} = 10,000$). Recommendation: $n_{boot} \geq 5,000$ for percentile method and $n_{boot} \geq 1,000$ for normal bootstrap
3. Use distribution of resampled statistics to build CI:
    - *Percentile method*:
        * Core idea: "If I have 10,000 bootstrap estimates, the middle 95% of those values form my 95% CI" 
        * Sort all bootstrap statistics
        * For (1-α)% CI, take the α/2 and (1-α/2) percentiles
        $$CI = [\hat{\theta}^*_{\alpha/2}, \hat{\theta}^*_{1 - \alpha/2}]$$
        where $\hat{\theta}^*_{\alpha/2} = (\alpha/2)\times 100th$ percentile of bootstrap distribution
    - *Normal approximation to bootstrap* (assumes the bootstrap distribution is approximately normal, i.e. $(\hat{\theta} - \theta) / SE_{boot} \sim \mathcal{N}(0,1)$ ):
        * Core idea: "Use bootstrap to estimate standard error, then construct CI using normal critical values (like traditional parametric CI)"
        * Estimate standard error from bootstrap distribution: $SE_{boot} = SD(\hat{\theta}^*)$
        * Use original sample statistic $\hat{\theta}$ and normal critical values
        $$CI = \hat{\theta} \pm z(\alpha/2) \times SE_{boot}$$
        where: $\hat{\theta}$ = statistic from original sample, $z(\alpha/2)$ = normal critical value (e.g., 1.96 for 95%), $SE_{boot} = \sqrt{\sum_{i}^{n_{boot}}(\hat{\theta}^*_i - mean(\hat{\theta}^*))^2 / (n_{boot}-1)}$ = standard deviation of bootstrap statistics

</div>

| | Percentile method | Normal approximation |
|---|---|---|
|Advantages| - Simple and intuitive - no complicated formulas </br> - Distribution-free - no normality assumption </br> - Automatically handles skewness - CI can be asymmetric </br> - Works for any statistic - median, percentiles, etc. </br> - Captures the actual bootstrap distribution shape | - Requires fewer bootstrap samples ($n_{boot} = 1,000$ often sufficient) </br> - Familiar form - looks like traditional CI </br> - Theoretically justified when CLT applies </br> - Computationally efficient - only need to estimate SE </br> - More stable for small $n_{boot}$ than percentile method |
|Disadvantages| - Requires many bootstrap samples ($n_{boot} \geq 5,000$ recommended) </br> - Can be unstable for small $n_{boot}$ or extreme percentiles </br> - May be too liberal (under-cover) for small samples </br> - Sensitive to outliers in original data | - Assumes normality - may be inappropriate for skewed distributions </br> - Always symmetric - can't capture asymmetry </br> - May perform poorly for highly skewed statistics </br> - Ignores shape of bootstrap distribution </br> - Can give impossible values (e.g., negative variance)|
|When to Use| - Your statistic's distribution is unknown or complex </br> - You expect the distribution to be skewed or asymmetric </br> - You have sufficient bootstrap samples ($n_{boot} \geq 5,000$) </br> - Default choice for most applications | - You believe the statistic is approximately normally distributed</br> - The sample size is large (CLT applies) </br> - You want computational efficiency (smaller $n_{boot}$) </br> - The bootstrap distribution appears roughly symmetric </br> - You're working with means or sums (CLT applies) |

In [ ]:
def bootstrap_percentile_ci(data, statistic_func, n_bootstrap=10000, confidence=0.95):
    """
    Calculate bootstrap CI using percentile method
    
    Parameters:
    -----------
    data : array-like
        Original data sample
    statistic_func : function
        Function to compute statistic (e.g., np.mean, np.median)
    n_bootstrap : int
        Number of bootstrap samples
    confidence : float
        Confidence level (e.g., 0.95 for 95% CI)
    
    Returns:
    --------
    ci_lower, ci_upper : float
        Lower and upper confidence bounds
    """
    n = len(data)
    bootstrap_statistics = []
    
    # Generate bootstrap samples
    for _ in range(n_bootstrap):
        # Resample with replacement
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        # Calculate statistic on this bootstrap sample
        bootstrap_stat = statistic_func(bootstrap_sample)
        bootstrap_statistics.append(bootstrap_stat)
    
    # Convert to array
    bootstrap_statistics = np.array(bootstrap_statistics)
    
    # Calculate percentiles
    alpha = 1 - confidence
    ci_lower = np.percentile(bootstrap_statistics, 100 * alpha/2)
    ci_upper = np.percentile(bootstrap_statistics, 100 * (1 - alpha/2))
    
    return ci_lower, ci_upper

In [ ]:
def bootstrap_normal_ci(data, statistic_func, n_bootstrap=1000, confidence=0.95):
    """
    Calculate bootstrap CI using normal approximation method
    
    Parameters:
    -----------
    data : array-like
        Original data sample
    statistic_func : function
        Function to compute statistic (e.g., np.mean, np.median)
    n_bootstrap : int
        Number of bootstrap samples
    confidence : float
        Confidence level (e.g., 0.95 for 95% CI)
    
    Returns:
    --------
    ci_lower, ci_upper : float
        Lower and upper confidence bounds
    """
    n = len(data)
    
    # Calculate statistic on original sample
    original_stat = statistic_func(data)
    
    # Generate bootstrap samples
    bootstrap_statistics = []
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        bootstrap_stat = statistic_func(bootstrap_sample)
        bootstrap_statistics.append(bootstrap_stat)
    
    # Convert to array
    bootstrap_statistics = np.array(bootstrap_statistics)
    
    # Estimate standard error from bootstrap distribution
    se_boot = np.std(bootstrap_statistics, ddof=1)
    
    # Get normal critical value
    from scipy import stats
    alpha = 1 - confidence
    z_critical = stats.norm.ppf(1 - alpha/2)
    
    # Construct symmetric CI
    ci_lower = original_stat - z_critical * se_boot
    ci_upper = original_stat + z_critical * se_boot
    
    return ci_lower, ci_upper


In [ ]:
# Example usage
data = np.array([45.2, 48.1, 43.8, 46.5, 44.9, 47.3, 45.8])

# CI for mean
mean_lower_perc, mean_upper_perc = bootstrap_percentile_ci(data, np.mean, n_bootstrap=10000, confidence=0.95)
print(f"95% CI for mean (percentile): [{mean_lower_perc:.2f}, {mean_upper_perc:.2f}]")

mean_lower_norm, mean_upper_norm = bootstrap_normal_ci(data, np.mean, n_bootstrap=1000, confidence=0.95)
print(f"95% CI for mean (normal): [{mean_lower_norm:.2f}, {mean_upper_norm:.2f}]")

# CI for median
median_lower_perc, median_upper_perc = bootstrap_percentile_ci(data, np.median, n_bootstrap=10000, confidence=0.95)
print(f"95% CI for median (percentile): [{median_lower_perc:.2f}, {median_upper_perc:.2f}]")

median_lower_norm, median_upper_norm = bootstrap_normal_ci(data, np.median, n_bootstrap=1000, confidence=0.95)
print(f"95% CI for median (normal): [{median_lower_norm:.2f}, {median_upper_norm:.2f}]")

Let's consider the following scenario on **Prediction Errors from a Model**: 

We analyse the data coming from a skewed distribution. We have a sample of 50 measures with the following statistics:

In [ ]:
np.random.seed(42)

# Original data (e.g., model errors)
# Skewed distribution - not normal
true_data = np.concatenate([
        np.random.exponential(2, 70),  # Most errors small
        np.random.exponential(10, 30)   # Some errors large
    ])

n = 50 # sample size

original_sample = np.random.choice(true_data, size=n, replace=False)

print(f"Sample size: n = {len(original_sample)}")
print(f"Sample median error: {np.median(original_sample):.2f}")
print(f"Sample mean error: {np.mean(original_sample):.2f}")

We would like to create a CI for the MEDIAN. There is no predefined formula. Moreover, the distribution is skewed (not normal). Standard methods don't apply. 

*Solution*: bootstrap

STEPS:

1. Resample n=50 original values WITH replacement from original sample
2. Compute median of resample
3. Repeat large number of times
4. Use distribution of bootstrap medians to build CI


In [ ]:
def demonstrate_bootstrap_ci():
    """Complete bootstrap demonstration"""
    np.random.seed(42)

    # Original data (e.g., model errors)
    # Skewed distribution - not normal!
    true_data = np.concatenate([
        np.random.exponential(2, 70),  # Most errors small
        np.random.exponential(10, 30)   # Some errors large
    ])

    original_sample = np.random.choice(true_data, size=50, replace=False)

    # Bootstrap procedure
    n_bootstrap = 10000
    bootstrap_medians = []
    bootstrap_means = []

    # Perform bootstrap
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(original_sample, size=len(original_sample), replace=True)
        bootstrap_medians.append(np.median(bootstrap_sample))
        bootstrap_means.append(np.mean(bootstrap_sample))

    bootstrap_medians = np.array(bootstrap_medians)
    bootstrap_means = np.array(bootstrap_means)

    # Compute CIs using different methods
    conf = 0.95 # confidence level
    alpha = 1 - conf
    alpha_05 = alpha / 2
    
    # Method 1: Percentile method (most common)
    ci_median_percentile = np.percentile(bootstrap_medians, [alpha_05*100, (1 - alpha_05)*100])
    ci_mean_percentile = np.percentile(bootstrap_means, [alpha_05*100, (1 - alpha_05)*100])

    # Method 2: Normal approximation to bootstrap
    median_estimate = np.median(original_sample)
    mean_estimate = np.mean(original_sample)
    se_median_boot = np.std(bootstrap_medians)
    se_mean_boot = np.std(bootstrap_means)
    z_crit = stats.norm.ppf(1 - alpha_05)

    ci_median_normal = [median_estimate - z_crit * se_median_boot,
                        median_estimate + z_crit * se_median_boot]
    ci_mean_normal = [mean_estimate - z_crit * se_mean_boot,
                    mean_estimate + z_crit * se_mean_boot]

    print("RESULTS:")
    print("-" * 80)
    print()
    print("FOR MEDIAN:")
    print(f"  Original estimate:        {median_estimate:.2f}")
    print(f"  Bootstrap SE:             {se_median_boot:.2f}")
    print(f"  95% CI (percentile):      [{ci_median_percentile[0]:.2f}, {ci_median_percentile[1]:.2f}]")
    print(f"  95% CI (normal approx):   [{ci_median_normal[0]:.2f}, {ci_median_normal[1]:.2f}]")
    print()
    print("FOR MEAN (for comparison):")
    print(f"  Original estimate:        {mean_estimate:.2f}")
    print(f"  Bootstrap SE:             {se_mean_boot:.2f}")
    print(f"  95% CI (percentile):      [{ci_mean_percentile[0]:.2f}, {ci_mean_percentile[1]:.2f}]")
    print(f"  95% CI (normal approx):      [{ci_mean_normal[0]:.2f}, {ci_mean_normal[1]:.2f}]")
    print()
    print("=" * 80)

    # Visualization
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

    # Plot 1: Original data
    ax1 = fig.add_subplot(gs[0, :])
    ax1.hist(original_sample, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
    ax1.axvline(np.median(original_sample), color='red', linewidth=3, 
            label=f'Median = {np.median(original_sample):.2f}')
    ax1.axvline(np.mean(original_sample), color='green', linewidth=3, linestyle='--',
            label=f'Mean = {np.mean(original_sample):.2f}')
    ax1.set_xlabel('Prediction Error', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Original Sample (n=50)\nSkewed distribution - not normal', 
                fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3, axis='y')

    # Plot 2: Bootstrap distribution for median
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.hist(bootstrap_medians, bins=50, density=True, alpha=0.7, 
            color='orange', edgecolor='black')

    # Add CI
    ax2.axvline(ci_median_percentile[0], color='red', linewidth=3, linestyle='--',
            label=f'95% CI: [{ci_median_percentile[0]:.2f}, {ci_median_percentile[1]:.2f}]')
    ax2.axvline(ci_median_percentile[1], color='red', linewidth=3, linestyle='--')
    ax2.axvline(median_estimate, color='darkred', linewidth=3,
            label=f'Original median = {median_estimate:.2f}')

    # Shade CI
    ax2.axvspan(ci_median_percentile[0], ci_median_percentile[1], 
            alpha=0.2, color='green')

    ax2.set_xlabel('Bootstrap Median', fontsize=12)
    ax2.set_ylabel('Density', fontsize=12)
    ax2.set_title(f'Bootstrap Distribution of MEDIAN\n({n_bootstrap:,} resamples)', 
                fontsize=13, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)

    # Plot 3: Bootstrap distribution for mean
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.hist(bootstrap_means, bins=50, density=True, alpha=0.7, 
            color='green', edgecolor='black')

    ax3.axvline(ci_mean_percentile[0], color='red', linewidth=3, linestyle='--',
            label=f'95% CI: [{ci_mean_percentile[0]:.2f}, {ci_mean_percentile[1]:.2f}]')
    ax3.axvline(ci_mean_percentile[1], color='red', linewidth=3, linestyle='--')
    ax3.axvline(mean_estimate, color='darkgreen', linewidth=3,
            label=f'Original mean = {mean_estimate:.2f}')

    ax3.axvspan(ci_mean_percentile[0], ci_mean_percentile[1], 
            alpha=0.2, color='green')

    ax3.set_xlabel('Bootstrap Mean', fontsize=12)
    ax3.set_ylabel('Density', fontsize=12)
    ax3.set_title(f'Bootstrap Distribution of MEAN\n({n_bootstrap:,} resamples)', 
                fontsize=13, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)

    plt.suptitle('Bootstrap Confidence Intervals', fontsize=16, fontweight='bold', y=0.98)
    plt.show()

In [ ]:
demonstrate_bootstrap_ci()

<div class="alert alert-warning">
<h4>💡 Key Insight: Bootstrap for CI: Advantages, Limitations, When to Use</h4>

<h5>Bootstrap Method Advantages:</h5>

1. Works for any statistic:
- No formula needed
- Median, percentiles, ratios, correlations, etc.
- Even complex ML metrics (F1-score, AUC, etc.)

2. No distribution assumptions:
- Doesn't require normality
- Works with skewed, multimodal data
- Robust to outliers (if statistic is robust)

3. Flexible:
- Different CI methods available
- Can adjust for bias
- Easy to implement


<h5>When to Use Bootstrap:</h5>

- Estimating CI for complex statistics (median, 90th percentile, etc.)
- Small samples where CLT doesn't apply
- Non-normal data
- When you don't know the theoretical distribution
- Cross-validation error estimates
- Ensemble model performance

<h5>Limitations:</h5>

- Computationally intensive (need many resamples)
- Assumes sample represents population well
- May underestimate CI width for very small $n$
- Not a magic fix for bad data

</div>

## ML Pipeline with CIs

<div class="alert alert-problem">
<h4>Real-World Application: Production Model Deployment Decision</h4>

**Business Context:**
You're deploying a ML model to detect fraudulent transactions. The business requires:
- Rigorous performance guarantees
- Regulatory requirements for uncertainty quantification
- High cost of false positives (customer friction) -> False Alarm Rate
- High cost of false negatives (fraud losses)
- Min recall 70% (Must detect at least 70% of fraud)
- Maximum 10% false alarm rate 10%
- At least, 30% precision 
    
Business Metrics:
- Fraud Detection Rate (*sensitivity*, fraction of relevant instances that were retrieved): $recall = \frac{TP}{TP + FN}$
- False Alarm Rate: $\frac{FP}{FP + TN}$

**Your Task:**
1. Train a model ([`RandomForestClassifier`][https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html]) and evaluate on test set
2. Compute CIs for precision and recall
3. Use CIs to make deployment decision
4. Create executive report with uncertainty quantification

**Learning Objectives:**
- Apply CIs to real ML metrics
- Make data-driven decisions with uncertainty
- Communicate results to non-technical stakeholders

</div>

In [ ]:
def generate_data(pos_class_rate=0.05, n_samples=5000):
    """
    Generates data for fraud detection with 20 features. Two possible classes: fraud transaction and normal transaction.

    Args:
        pos_class_rate (float, optional): Positive class rate for binary classification. Defaults to 0.05.
        n_samples (int, optional): Number of samples in the generated dataset

    Returns:
        X, y - feature values and target class
    """
    from sklearn.datasets import make_classification
    
    # Generate synthetic fraud detection dataset
    np.random.seed(42)
    
    # Imbalanced dataset (fraud is rare)
    X, y = make_classification(
        n_samples=n_samples,
        n_features=20,
        n_informative=15,
        n_redundant=5,
        n_classes=2,
        weights=[1-pos_class_rate, pos_class_rate],  
        flip_y=0.01,
        random_state=42
    )
    
    return X, y

In [ ]:
def train_test_val_split(train_size=0.7, test_size=0.5):
    """
    Generates data for fraud detection. Performs train, val, test split. 

    Args:
        train_size (float, optional): Train set size rate. Defaults to 0.7.
        test_size (float, optional): Test set size rate from the remaining data after train split. Defaults to 0.5.

    Returns:
        X_train, y_train, X_val, y_val, X_test, y_test
    """
    
    from sklearn.model_selection import train_test_split
    
    # Imbalanced dataset (fraud is rare)
    X, y = generate_data(pos_class_rate=0.05, n_samples=5000)
    
    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=1-train_size, random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=test_size, random_state=42, stratify=y_temp
    )
    
    print(f"Dataset sizes:")
    print(f"  Training:   {len(X_train):,} samples")
    print(f"  Validation: {len(X_val):,} samples")
    print(f"  Test:       {len(X_test):,} samples")
    print()
    
    print(f"Class distribution (test set):")
    print(f"  Legitimate: {np.sum(y_test == 0):,} ({np.mean(y_test == 0):.1%})")
    print(f"  Fraud:      {np.sum(y_test == 1):,} ({np.mean(y_test == 1):.1%})")
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
# generate data
X_train, y_train, X_val, y_val, X_test, y_test = train_test_val_split(train_size=0.7, test_size=0.5)

In [ ]:
def train_model_get_pred(X_train, y_train, X_val, y_val, X_test):
    """Trains RandomForestClassifier on the provided data and makes prediction for X_test.

    Args:
        X_train (_type_): X_train feature set used for training
        y_train (_type_): y_train targets set used for training
        X_val (_type_): validation feature set
        y_val (_type_): validation target set
        X_test (_type_): X test set used for prediction

    Returns:
        y_pred: class predictions for X_test
        y_pred_proba: class probabilities for X_test
    """
    
    from sklearn.ensemble import RandomForestClassifier
    
    # Train model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        class_weight='balanced'  # Handle imbalance
    )
    
    model.fit(X_train, y_train)
        
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    return model, y_pred, y_pred_proba
    

In [ ]:
def calculate_metrics(y_test, y_pred, y_pred_proba):
    """
    Calculates accuracy-based metrics given y_test (true target values) and predicted values.
    """
    
    from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                                 f1_score, roc_auc_score, confusion_matrix,
                                 classification_report, roc_curve)
    
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    print("Point Estimates (Test Set):")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  AUC-ROC:   {auc:.4f}")
    print()
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print("Confusion Matrix:")
    print(f"                 Predicted")
    print(f"                 Legit    Fraud")
    print(f"Actual Legit     {tn:<8d} {fp:<8d}")
    print(f"Actual Fraud     {fn:<8d} {tp:<8d}")
    print()
    
    # Business metrics
    fraud_detection_rate = recall
    false_alarm_rate = fp / (fp + tn)
    
    print("Business Metrics:")
    print(f"  Fraud Detection Rate: {fraud_detection_rate:.1%}")
    print(f"  False Alarm Rate:     {false_alarm_rate:.1%}")
    
    return accuracy, precision, recall, f1, auc, cm, fraud_detection_rate, false_alarm_rate

In [ ]:
# Train model and get predictions 
model, y_pred, y_pred_proba = train_model_get_pred(X_train, y_train, X_val, y_val, X_test)    
    
    
# Calculate metrics
accuracy, precision, recall, f1, auc, cm, fraud_detection_rate, false_alarm_rate = calculate_metrics(y_test, y_pred, y_pred_proba)

<div class="alert alert-exercise">
<h4>QUESTION: Calculate Bootstrap Metrics</h4>

Right a python function that calculates accuracy-based metrics in a bootstrap procedure. 

```
def calculate_bootstrap_metrics(y_test, y_pred, y_pred_proba, n_bootstrap=5000):
    """
    Calculates bootstrap metrics and returns a dictionary of values.
    
    Args:
        y_test (np.array): true targets for test data
        y_pred (np.array): predicted targets for test data
        y_pred_proba (np.array): predicted class probabilities for test data
        n_bootstrap (int, optional): number of bootstraps to perform. Defaults to 5000.

    Returns:
        Dictionary containing the bootstrap values for accuracy, precision, recall, f1, auc, 
        fraud_detection_rate, false_alarm_rate
    """    
```

</div>

In [ ]:
# ANSWER
def calculate_bootstrap_metrics(y_test, y_pred, y_pred_proba, n_bootstrap=5000):
    """
    Calculates bootstrap metrics and returns a dictionary of values.
    
    Args:
        y_test (np.array): true targets for test data
        y_pred (np.array): predicted targets for test data
        y_pred_proba (np.array): predicted class probabilities for test data
        n_bootstrap (int, optional): number of bootstraps to perform. Defaults to 5000.

    Returns:
        Dictionary containing the bootstrap values for accuracy, precision, recall, f1, auc, 
        fraud_detection_rate, false_alarm_rate
    """    
    
    from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                                 f1_score, roc_auc_score, confusion_matrix)
    
    
    # Storage
    bootstrap_metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'auc': [],
        'fraud_detection_rate': [],
        'false_alarm_rate': []
    }
    
    # YOU CODE HERE
        
    return bootstrap_metrics
    

In [ ]:
n_bootstrap = 5000
confidence_level = 0.95
    
print(f"Bootstrap parameters:")
print(f"  Samples: n_boot = {n_bootstrap:,}")
print(f"  Confidence: {confidence_level:.0%}")
    
    
# calculate bootstrap metrics
bootstrap_metrics = calculate_bootstrap_metrics(y_test, y_pred, y_pred_proba, n_bootstrap=n_bootstrap)

<div class="alert alert-exercise">
<h4>QUESTION: Calculate Bootstrap CI</h4>

Write a python function that calculates bootstrap CI for accuracy metrics using percentile method.

```
def bootstrap_ci(data, confidence=0.95):
    """
    Calculates bootstrap CI using percentile method.

    Args:
        data (np.array): sample data
        confidence (float, optional): confidence level. Defaults to 0.95.

    Returns:
        lower and upper bounds of CI
    """
```

</div>

In [ ]:
# ANSWER
def bootstrap_ci(data, confidence=0.95):
    """
    Calculates bootstrap CI using percentile method.

    Args:
        data (np.array): sample data
        confidence (float, optional): confidence level. Defaults to 0.95.

    Returns:
        lower and upper bounds of CI
    """
    
    pass

In [ ]:
# Calculate CIs    
cis = {metric: bootstrap_ci(values) for metric, values in bootstrap_metrics.items()}
    
print(f"{'Metric':<25s} {'Estimate':<12s} {'95% CI Lower':<15s} {'95% CI Upper':<15s} {'Width':<12s}")
print("-" * 90)
    
metrics_display = [
        ('accuracy', 'Accuracy', accuracy),
        ('precision', 'Precision', precision),
        ('recall', 'Recall', recall),
        ('f1', 'F1-Score', f1),
        ('auc', 'AUC-ROC', auc),
        ('fraud_detection_rate', 'Fraud Detection Rate', fraud_detection_rate),
        ('false_alarm_rate', 'False Alarm Rate', false_alarm_rate)
    ]
    
for key, name, est in metrics_display:
    ci = cis[key]
    width = ci[1] - ci[0]
    print(f"{name:<25s} {est:<12.4f} {ci[0]:<15.4f} {ci[1]:<15.4f} {width:<12.4f}")
    

In [ ]:
def business_reqs_check(cis, min_recall, max_false_alarm, min_precision, recall, false_alarm_rate, precision):
    
    print("BUSINESS REQUIREMENTS:")
    print(f"  ✓ Fraud Detection Rate ≥ {min_recall:.0%} (high recall)")
    print(f"  ✓ False Alarm Rate ≤ {max_false_alarm:.0%} (low false positives)")
    print(f"  ✓ Precision ≥ {min_precision:.0%} (accuracy when flagging fraud)")
    print()
    
    # Check requirements using LOWER bounds of CIs (conservative)
    recall_lower = cis['recall'][0]
    fpr_upper = cis['false_alarm_rate'][1]  # Use upper bound (worst case)
    precision_lower = cis['precision'][0]
    
    meets_recall = recall_lower >= min_recall
    meets_fpr = fpr_upper <= max_false_alarm
    meets_precision = precision_lower >= min_precision
    
    req_check = f"""
    EVALUATION (using CI bounds for conservative assessment):
    
    Recall Requirement:
      Point estimate: {recall:.1%}
      95% CI: [{cis['recall'][0]:.1%}, {cis['recall'][1]:.1%}]
      Lower bound ({recall_lower:.1%}) {'≥' if meets_recall else '<'} {min_recall:.0%}
      Status: {'✓ PASS' if meets_recall else '✗ FAIL'}
        
    False Alarm Rate Requirement:
      Point estimate: {false_alarm_rate:.1%}
      95% CI: [{cis['false_alarm_rate'][0]:.1%}, {cis['false_alarm_rate'][1]:.1%}]
      Upper bound ({fpr_upper:.1%}) {'≤' if meets_fpr else '>'} {max_false_alarm:.0%}
      Status: {'✓ PASS' if meets_fpr else '✗ FAIL'}
    
    Precision Requirement:
      Point estimate: {precision:.1%}
      95% CI: [{cis['precision'][0]:.1%}, {cis['precision'][1]:.1%}]
      Lower bound ({precision_lower:.1%}) {'≥' if meets_precision else '<'} {min_precision:.0%}
      Status: {'✓ PASS' if meets_precision else '✗ FAIL'}
    """
    
    #print(req_check)
    
    # Overall decision
    all_pass = meets_recall and meets_fpr and meets_precision
    
    
    req_check = req_check + f"""
    DEPLOYMENT DECISION: {'✓ APPROVE' if all_pass else '✗ DO NOT APPROVE'}
    """
    
    
    if all_pass:
        print("The model meets all business requirements with 95% confidence.")
        print("Recommendation: PROCEED TO PRODUCTION")
    else:
        print("The model does not meet all requirements.")
        print("Recommendation: IMPROVE MODEL OR ADJUST REQUIREMENTS")
        
    return req_check, all_pass
    

In [ ]:
recall_lower = cis['recall'][0]
fpr_upper = cis['false_alarm_rate'][1]  # Use upper bound (worst case)
precision_lower = cis['precision'][0]

In [ ]:
# Business requirements
min_recall = 0.70  # Must detect at least 70% of fraud
max_false_alarm = 0.10  # Maximum 10% false alarm rate
min_precision = 0.30  # At least 30% precision
    
req_check, all_pass = business_reqs_check(cis, min_recall, max_false_alarm, min_precision, recall, false_alarm_rate, precision)

print(req_check)

In [ ]:
from sklearn.metrics import roc_curve

# visualisation
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(4, 3, hspace=0.4, wspace=0.3)

# Plot 1: Confusion Matrix
ax1 = fig.add_subplot(gs[0, 0])

cm_display = cm.astype(float)
im = ax1.imshow(cm_display, cmap='Blues', alpha=0.7)

for i in range(2):
    for j in range(2):
        text = ax1.text(j, i, f'{cm[i, j]}\n({cm[i,j]/np.sum(cm[i,:]):.1%})',
                        ha="center", va="center", color="black",
                        fontsize=14, fontweight='bold')

ax1.set_xticks([0, 1])
ax1.set_yticks([0, 1])
ax1.set_xticklabels(['Predicted\nLegitimate', 'Predicted\nFraud'])
ax1.set_yticklabels(['Actual\nLegitimate', 'Actual\nFraud'])
ax1.set_title('Confusion Matrix\n(Test Set Performance)', 
                fontsize=13, fontweight='bold')

# Plot 2: ROC Curve
ax2 = fig.add_subplot(gs[0, 1])

fpr_roc, tpr_roc, _ = roc_curve(y_test, y_pred_proba)

ax2.plot(fpr_roc, tpr_roc, linewidth=3, color='darkorange',
        label=f'ROC (AUC = {auc:.3f})')
ax2.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random')

# Mark operating point
ax2.plot([false_alarm_rate], [recall], 'ro', markersize=15,
        label=f'Operating Point\n(FPR={false_alarm_rate:.3f}, TPR={recall:.3f})')

ax2.set_xlabel('False Positive Rate', fontsize=12)
ax2.set_ylabel('True Positive Rate (Recall)', fontsize=12)
ax2.set_title('ROC Curve', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Key Metrics with CIs
ax3 = fig.add_subplot(gs[0, 2])

metrics_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
estimates_plot = [accuracy, precision, recall, f1]
cis_plot = [cis['accuracy'], cis['precision'], cis['recall'], cis['f1']]
colors_plot = ['steelblue', 'green', 'orange', 'purple']

y_pos = np.arange(len(metrics_plot))

for i, (metric, est, ci, color) in enumerate(zip(metrics_plot, estimates_plot, cis_plot, colors_plot)):
    ax3.barh(y_pos[i], ci[1] - ci[0], left=ci[0], height=0.4,
            alpha=0.6, color=color)
    ax3.plot(est, y_pos[i], 'o', markersize=15, color='red',
            markeredgecolor='darkred', markeredgewidth=2, zorder=5)

ax3.set_yticks(y_pos)
ax3.set_yticklabels(metrics_plot)
ax3.set_xlabel('Value', fontsize=12)
ax3.set_title('Performance Metrics with 95% CIs\n(Red = point estimate)',
                fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')
ax3.set_xlim(0, 1)

# Plots 4-7: Bootstrap distributions for key metrics
metrics_boot = [('Accuracy', 'accuracy'), ('Precision', 'precision'),
                ('Recall', 'recall'), ('F1-Score', 'f1')]

for idx, (name, key) in enumerate(metrics_boot):
    ax = fig.add_subplot(gs[1, idx if idx < 3 else idx-3])
    
    data = np.array(bootstrap_metrics[key])
    est = estimates_plot[idx]
    ci = cis[key]
    
    ax.hist(data, bins=50, alpha=0.6, color=colors_plot[idx], edgecolor='black')
    ax.axvline(est, color='red', linewidth=3, label=f'Estimate: {est:.3f}')
    ax.axvline(ci[0], color='darkred', linewidth=2, linestyle='--')
    ax.axvline(ci[1], color='darkred', linewidth=2, linestyle='--')
    
    ax.set_xlabel(name, fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'Bootstrap Distribution: {name}',
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

# Plot 8: Business Metrics
ax8 = fig.add_subplot(gs[2, 0])

business_names = ['Fraud\nDetection\nRate', 'False\nAlarm\nRate']
business_ests = [fraud_detection_rate, false_alarm_rate]
business_cis = [cis['fraud_detection_rate'], cis['false_alarm_rate']]
business_colors = ['green', 'red']

y_pos_bus = np.arange(len(business_names))

for i, (name, est, ci, color) in enumerate(zip(business_names, business_ests, business_cis, business_colors)):
    ax8.barh(y_pos_bus[i], ci[1] - ci[0], left=ci[0], height=0.4,
            alpha=0.6, color=color)
    ax8.plot(est, y_pos_bus[i], 'o', markersize=15, color='darkblue',
            markeredgecolor='black', markeredgewidth=2, zorder=5)

# Add requirement lines
ax8.axvline(min_recall, color='green', linewidth=2, linestyle=':',
            label=f'Min Required: {min_recall:.0%}')
ax8.axvline(max_false_alarm, color='red', linewidth=2, linestyle=':',
            label=f'Max Allowed: {max_false_alarm:.0%}')

ax8.set_yticks(y_pos_bus)
ax8.set_yticklabels(business_names)
ax8.set_xlabel('Rate', fontsize=12)
ax8.set_title('Business-Critical Metrics with Requirements',
                fontsize=13, fontweight='bold')
ax8.legend(fontsize=10)
ax8.grid(True, alpha=0.3, axis='x')
ax8.set_xlim(0, 1)

# Plot 9: Requirements Check
ax9 = fig.add_subplot(gs[2, 1])
ax9.axis('off')

ax9.text(0.5, 0.3, req_check, fontsize=10, family='monospace',
        verticalalignment='center', horizontalalignment='center',
        bbox=dict(boxstyle='round',
                    facecolor='lightgreen' if all_pass else 'lightcoral',
                    alpha=0.4))

# Plot 10: Cost-Benefit Analysis
ax10 = fig.add_subplot(gs[2, 2])

# Assuming costs
cost_fp = 50  # Cost per false positive (customer friction)
cost_fn = 1000  # Cost per false negative (fraud loss)
n_transactions_monthly = 100000
fraud_rate = 0.05

# Expected costs
expected_fp = n_transactions_monthly * (1 - fraud_rate) * false_alarm_rate
expected_fn = n_transactions_monthly * fraud_rate * (1 - recall)

monthly_cost_fp = expected_fp * cost_fp
monthly_cost_fn = expected_fn * cost_fn
total_monthly_cost = monthly_cost_fp + monthly_cost_fn

costs = ['False Positives\n(Customer Friction)', 'False Negatives\n(Fraud Losses)']
cost_values = [monthly_cost_fp, monthly_cost_fn]
cost_colors = ['orange', 'red']

bars = ax10.bar(costs, cost_values, color=cost_colors, alpha=0.7,
                edgecolor='black', linewidth=2)

# Add value labels
for bar, value in zip(bars, cost_values):
    height = bar.get_height()
    ax10.text(bar.get_x() + bar.get_width()/2., height,
                f'${value:,.0f}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')

ax10.set_ylabel('Monthly Cost ($)', fontsize=12)
ax10.set_title(f'Expected Monthly Costs\nTotal: ${total_monthly_cost:,.0f}',
                fontsize=13, fontweight='bold')
ax10.grid(True, alpha=0.3, axis='y')

# Plot 11: Production Monitoring Plan
ax11 = fig.add_subplot(gs[3, :])
ax11.axis('off')

monitoring_plan = f"""
PRODUCTION MONITORING PLAN WITH CI-BASED ALERTS
═══════════════════════════════════════════════════════════════════════════════════════════════════

ALERT THRESHOLDS (based on 95% Confidence Intervals):

┌─────────────────────┬───────────────┬───────────────────┬───────────────────┬─────────────────┐
│ Metric              │ Point Est     │ CI Lower (95%)    │ Alert Threshold   │ Action          │
├─────────────────────┼───────────────┼───────────────────┼───────────────────┼─────────────────┤
│ Recall              │ {recall:>6.1%}        │ {recall_lower:>6.1%}            │ < {recall_lower:.1%}          │ CRITICAL         │
│ Precision           │ {precision:>6.1%}        │ {precision_lower:>6.1%}            │ < {precision_lower:.1%}          │ WARNING        │
│ False Alarm Rate    │ {false_alarm_rate:>6.1%}        │ {fpr_upper:>6.1%}            │ > {fpr_upper:.1%}            │ CRITICAL        │
│ F1-Score            │ {f1:>6.1%}        │ {cis['f1'][0]:>6.1%}            │ < {cis['f1'][0]:.1%}            │ WARNING        │
└─────────────────────┴───────────────┴───────────────────┴───────────────────┴─────────────────┘

═══════════════════════════════════════════════════════════════════════════════════════════════════

EXPECTED PERFORMANCE RANGES (with 95% confidence):
• Accuracy: [{cis['accuracy'][0]:.1%}, {cis['accuracy'][1]:.1%}]
• Precision: [{cis['precision'][0]:.1%}, {cis['precision'][1]:.1%}]
• Recall: [{cis['recall'][0]:.1%}, {cis['recall'][1]:.1%}]
• F1-Score: [{cis['f1'][0]:.1%}, {cis['f1'][1]:.1%}]

Performance within these ranges is EXPECTED and NORMAL.
Performance outside these ranges requires INVESTIGATION.

═══════════════════════════════════════════════════════════════════════════════════════════════════
"""

ax11.text(0.5, 0.1, monitoring_plan, fontsize=8.5, family='monospace',
            verticalalignment='center', horizontalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.suptitle('ML Pipeline Executive Dashboard: Fraud Detection Model with Confidence Intervals',
            fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

We can imagine the following monitoring strategy:
    
1. REAL-TIME MONITORING:
    - Track all metrics on rolling 24-hour windows
    - Compare against alert thresholds
    - Automatic alerts when thresholds breached
    
2. WEEKLY REVIEW:
    - Recompute bootstrap CIs on recent data
    - Check for distribution drift
    - Update thresholds if necessary
    
3. MONTHLY AUDIT:
    - Full model re-evaluation
    - Business impact assessment
    - Cost-benefit analysis update
    - Decision: continue, retrain, or replace
    
4. ESCALATION PROTOCOL:
    - WARNING: Notify ML team, investigate within 48h
    - CRITICAL: Page on-call engineer, investigate immediately
    - Multiple alerts: Consider emergency rollback

<div class="alert alert-danger">
<h4>⚠️ Common Misconception: K-Fold CV ≠ Confidence Intervals</h4>
<p><strong>The Mistake:</strong> Many practitioners report k-fold cross-validation results as "mean ± std" and treat this as a confidence interval. This is statistically incorrect!</p>

<p><strong>Why K-Fold CV Doesn't Give Proper CIs:</strong></p>
<ul>
    <li><strong>Dependence Problem:</strong> The k scores are <em>not independent</em> — training sets overlap heavily between folds (each data point appears in k-1 training sets)</li>
    <li><strong>Underestimated Variance:</strong> Because of correlation, the standard error calculated from k folds is biased downward</li>
    <li><strong>No Coverage Guarantees:</strong> You can't claim "95% confidence" using the normal formula — the actual coverage may be far from 95%</li>
</ul>

<p><strong>What Each Method Is Actually For:</strong></p>
<table style="width:100%; margin: 10px 0; border-collapse: collapse;">
    <tr style="background-color: rgba(0,0,0,0.05);">
        <th style="padding: 8px; text-align: left; border: 1px solid #ddd;"></th>
        <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">K-Fold Cross-Validation</th>
        <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Bootstrap CI</th>
    </tr>
    <tr>
        <td style="padding: 8px; border: 1px solid #ddd;"><strong>Purpose</strong></td>
        <td style="padding: 8px; border: 1px solid #ddd;">Model selection & comparison</td>
        <td style="padding: 8px; border: 1px solid #ddd;">Uncertainty quantification with statistical guarantees</td>
    </tr>
    <tr style="background-color: rgba(0,0,0,0.02);">
        <td style="padding: 8px; border: 1px solid #ddd;"><strong>What you get</strong></td>
        <td style="padding: 8px; border: 1px solid #ddd;">Robust estimate of expected performance</td>
        <td style="padding: 8px; border: 1px solid #ddd;">Range with known coverage probability</td>
    </tr>
    <tr>
        <td style="padding: 8px; border: 1px solid #ddd;"><strong>Valid reporting</strong></td>
        <td style="padding: 8px; border: 1px solid #ddd;">"Mean accuracy: 0.85 ± 0.03"</td>
        <td style="padding: 8px; border: 1px solid #ddd;">"95% CI: [0.82, 0.88]"</td>
    </tr>
</table>

<p><strong>Best Practice in ML Pipelines:</strong></p>
<ol>
    <li><strong>During development:</strong> Use k-fold CV to compare and select models</li>
    <li><strong>For final reporting:</strong> After choosing your model, use bootstrap on the held-out test set to get proper confidence intervals for performance metrics</li>
    <li><strong>Never:</strong> Don't treat k-fold CV standard deviation as if it gives you confidence intervals</li>
</ol>

<p style="margin-top: 15px;"><strong>💡 Remember:</strong> K-fold CV helps you make better decisions about which model to choose. Bootstrap CIs help you communicate the uncertainty in your final model's performance with statistical rigor.</p>
</div>

## Return to the Opening Problem

Original scenario:
- Reported model accuracy: 94.2%
- Reality in production: 92.5%

**Need for uncertainty quantification**

In [ ]:
# Simulate the original situation
n_test = 500
observed_correct = 471  # 94.2%
p_hat = observed_correct / n_test # proportion point estimate 

print(f"Proportion point estimate: {p_hat}")

What should have been done:

1. Compute CI for proportion

Let's consider 95% confidence level. Therefore, $\alpha = 1 - 0.95 = 0.05 \Rightarrow \alpha/2 = 0.025 \Rightarrow z_{\alpha/2} = 1.96$.

$SE = \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = \sqrt{\frac{0.942(1 - 0.942)}{500}} = \sqrt{0.0001} = 0.01$

$$\begin{array}{l}\hat{p} - z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} = 0.942 - 1.96 \times 0.01 = 0.9224\\ \hat{p} + z_{\alpha/2}\sqrt{\frac{\hat{p}(1 - \hat{p})}{n}} =  0.642 + 1.96 \times 0.01 = 0.962\end{array}$$


In Python, we can also use [statsmodels.stats.proportion.proportion_confint](https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html) to calculate a confidence interval for a binomial proportion.

In [ ]:
from statsmodels.stats.proportion import proportion_confint # CI for binomial proportions

In [ ]:
conf = 0.95
ci = proportion_confint(observed_correct, n_test, alpha=1-conf, method='normal')
    
print(f"CI for proportion:")
print(f"   Test set: n = {n_test}")
print(f"   Observed accuracy: {p_hat:.1%}")
print(f"   95% CI: [{ci[0]:.1%}, {ci[1]:.1%}]")

2. Report to stakeholders:

- Our model achieved 94.2% accuracy on the test set.
- With 95% confidence, the true accuracy is between 92.2% and 96.2%.
- In production, we should expect performance in this range.

3. Set expectations:
- Worst case: accuracy could be as low as 92.2%
- If our minimum requirement is 85%, it still exceeds it.

4. Observed in production: 92.2%
- Within the predicted range. No crisis.
- Stakeholders were properly informed of uncertainty.

**Lessons learned:**

- Always report uncertainty, not just point estimates
- Use CIs to set realistic expectations
- Design monitoring systems around CI bounds
- Your credibility depends on honest uncertainty quantification


## Common Mistakes

<div class="alert alert-danger">
<h4>⚠️ Common Pitfalls</h4>

- Misinterpreting CI as probability statement
- Using normal approximation for small samples
- Forgetting to check assumptions
- Reporting point estimate without CI
- Confusing statistical and practical significance
- Using wrong method for proportions (use Wilson)

</div>

## ML Applications

<div class="alert alert-primary">
<h4>🤖 ML Applications</h4>

- Model Evaluation: Always report metrics with CIs
- A/B Testing: Compare models using CI for difference
- Production Monitoring: Set alert thresholds based on CI bounds
- Stakeholder Communication: Honest reporting of uncertainty
- Risk Management: Worst-case scenarios from CI bounds

</div>

## Key Takeaways

<div class="alert alert-summary">
<h4>🎓 Key Takeaways</h4>

1. Confidence Intervals:

- Range of plausible values for unknown parameter
- Correct interpretation: "95% of such intervals contain $\theta$" (frequentist)
- NOT: "95% probability $\theta$ is in this interval"
- Components: Point estimate ± Margin of error

2. Construction Methods:

Pivot method: Use quantity with known distribution
Normal approximation: For means (use t-distribution when σ unknown)
Bootstrap: For any statistic, no assumptions needed
Special formulas: Proportions, differences, etc.

3. Interpreting CIs:

- Width indicates precision
- Wider CI = more uncertainty
- Factors: sample size, variability, confidence level
- CI overlapping zero = no significant difference

4. Key Formulas:

- Mean ($\sigma$ unknown):
$$\bar{X} \pm t_{\alpha/2, n-1} \times \frac{s}{\sqrt{n}}$$
- Proportion:
$$\hat{p} \pm z_{\alpha/2} \times \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$$
- Variance:
- Bootstrap percentile:
$[Q_{\alpha/2}, Q_{1-\alpha/2}]$ of bootstrap distribution

</div>

## Useful Links

1. [Confidence Intervals, Clearly Explained!!! by StatQuest](https://www.youtube.com/watch?v=TqOeMYtOc1w)
2. [What are degrees of freedom? by James Gilbert](https://www.youtube.com/watch?v=rATNoxKg1yA)